In [99]:
import time
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler
from collections import defaultdict

In [ ]:
class Watcher:
    DIRECTORY_TO_WATCH = "07_Lief"


    def __init__(self):
        self.observer = Observer()

    def run(self):
        patt = ['*.sdlrpx', '*.sdlxliff', '*.wsxz']
        event_handler = Processor(patterns=patt) ## This will handle all events from watchdog
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=True)
        self.observer.start()
        try:
            while True:
                time.sleep(5)
        except:
            self.observer.stop()
            print("Error")

            self.observer.join()


class Processor(PatternMatchingEventHandler):
    @staticmethod
    def on_created(event):
        
        mt_providers = Processor.check_for_mt(event.src_path)
        if len(mt_providers) > 0:
            Processor.print_warning(os.path.dirname(event.src_path), mt_providers)

    @classmethod
    def check_for_mt(cls, file_path):
        path = os.path.join(os.path.dirname(os.path.normpath(file_path)), '..', '02_vor')
        files = glob.iglob(path+'\**\*.sdlxliff', recursive=True)
        
        providers = ['DeepL Translator provider using DeepL Translator ']
        string = 'origin-system="'
        mt_providers = defaultdict(dict)
        
        for file in files:
            for provider in providers:
                if string+provider in open(file, encoding='utf-8').read():
                    mt_providers[file][provider] = True
        
        return mt_providers
    
    def print_warning(file_path, mt_providers):
        
        try:
            with open(os.path.join(file_path, "MT WARNING.txt"), "w+") as w:
                
                for keys, values in mt_providers.items():
                    w.write(str('MT found in file {}'.format(keys)))
                    
                    for provider in values.keys():
                        w.write(str('\nProvider: {}'.format(provider)))
        
        except KeyboardInterrupt:
                raise

if __name__ == '__main__':
    w = Watcher()
    w.run()